In [1]:
# !nvidia-smi
# !pip install einops
# !pip install pyyaml==5.4.1
# !pip install transformers
# !pip install git+https://github.com/neelnanda-io/Easy-Transformer.git

zsh:1: command not found: nvidia-smi
  Using cached PyYAML-5.4.1-cp310-cp310-macosx_11_0_arm64.whl
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
  Cloning https://github.com/neelnanda-io/Easy-Transformer.git to /private/var/folders/yj/gj5_5c_j1399_7h47zvzddtc0000gn/T/pip-req-build-rtmu0yxb
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/Easy-Transformer.git /private/var/folders/yj/gj5_5c_j1399_7h47zvzddtc0000gn/T/pip-req-build-rtmu0yxb
  Resolved https://github.com/neelnanda-io/Easy-Transformer.git to commit 99e8a599f244dcdd70a5cf7005b47c7a057f4681
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached jaxtyping-0.2.14-py3-none-any.whl (20 kB)
  Using cached rich-13.3.3-py3-none-any.whl (238 kB)
  Using cached transformers-4.27.4-py3-none-any.whl (6.8 MB)


In [1]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
import tqdm.notebook as tqdm

import random
import time

from pathlib import Path
import pickle
import os


import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import plotly.io as pio
#pio.renderers.default = "colab"
import plotly.graph_objects as go

from torch.utils.data import DataLoader

from functools import *
import pandas as pd
import gc
import collections
import copy

# import comet_ml
import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses


In [6]:
## pretty sure easy_transformer was obsolete, so needed to reformat things a bit from Neel's demo
import transformer_lens as tl
from transformer_lens.utils import gelu_new, to_numpy, get_corner #helper functions
from transformer_lens.hook_points import HookedRootModule, HookPoint
#from transformer_lens import EasyTransformer 
#from transformer_lens.EasyTransformer import TransformerBlock, MLP, Attention, LayerNormPre, PosEmbed, Unembed, Embed
# from easy_transformer.experiments import ExperimentMetric, AblationConfig, EasyAblation, EasyPatching, PatchingConfig
dir(transformer_lens.EasyTransformer)


['OV',
 'QK',
 'T_destination',
 'W_E',
 'W_E_pos',
 'W_K',
 'W_O',
 'W_Q',
 'W_U',
 'W_V',
 'W_in',
 'W_out',
 'W_pos',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_call_impl',
 '_get_backward_hooks',
 '_get_backward_pre_hooks',
 '_get_name',
 '_load_from_state_dict',
 '_maybe_warn_non_full_backward_hook',
 '_named_members',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_replicate_for_data_parallel',
 '_save_to_state_dict',
 '_slow_forward',
 '_version',
 'accumulated_bias',
 'add_caching_hooks',
 'add_hook',
 'add_module',
 'add_perma_hook',
 'all_composition_sc

HOOK POINTS
wrap around the activation in a network, using PyTorch hooks to access and edit them.
- The root module is the model we want to study
- The HookedRootModule class is inherited by the model
- using "run_with_hooks" function, you can pass a list of [hook, layer] through the model (forward pass)
- each hook is a function of the activation and the hookpoint class attached to that function (I don't understand this yet)

As an example, they look at a toy, two-layer network, where each layer consists of: 
input (x) ->  square (x**2) -> translate (+=const) = output
- Hook points are wrapped around the input, square, and output parts of each layer



In [34]:

class SquareThenAdd(nn.Module):
    def __init__(self, offset):
        super().__init__()
        self.offset = nn.Parameter(torch.tensor(offset))
        self.hook_square = HookPoint()
    
    def forward(self, x):
        # The hook_square doesn't change the value, but lets us access it
        square = self.hook_square(x * x)
        return self.offset + square
    
class TwoLayerModel(HookedRootModule):
    def __init__(self):
        super().__init__()
        self.layer1 = SquareThenAdd(3.)
        self.layer2 = SquareThenAdd(-4.)
        self.hook_in = HookPoint()
        self.hook_mid = HookPoint()
        self.hook_out = HookPoint()

        # We need to call the setup function of HookedRootModule to build an 
        # internal dictionary of modules and hooks, and to give each hook a name
        super().setup()
    
    def forward(self, x):
        # We wrap the input and each layer's output in a hook - they leave the 
        # value unchanged (unless there's a hook added to explicitly change it), 
        # but allow us to access it.
        x_in = self.hook_in(x)
        x_mid = self.hook_mid(self.layer1(x_in))
        x_out = self.hook_out(self.layer2(x_mid))
        return x_out
model = TwoLayerModel()

In [35]:
## save the activations at each hook point in a cache

#### figure out how run_with_cache works ... I think we can just do it later when getting the logits

cache = {}
model.cache_all(cache)
#model.run_with_cache(cache)
print('Model output:', model(torch.tensor(5.)).item())
for key in cache:
    print(f"Value cached at hook {key}", cache[key].item())
model.reset_hooks()

Model output: 780.0
Value cached at hook hook_in 5.0
Value cached at hook layer1.hook_square 25.0
Value cached at hook hook_mid 28.0
Value cached at hook layer2.hook_square 784.0
Value cached at hook hook_out 780.0


In [46]:
### make a function to set the square value of layer2 to 0, and show the output
model.reset_hooks()
def set_to_zero_hook(tensor, hook):
    print(hook.name)
    return torch.tensor(0.)
print('Output after intervening on layer2.hook_scaled', 
      model.run_with_hooks(torch.tensor(5.),
                           fwd_hooks = [('layer2.hook_square', set_to_zero_hook)], reset_hooks_end = False).item())
## A better way to do the cache, but there's no way to insert the hooks here and run it all at the same time? 
### I think what this does is calls "get_caching_hooks" on the model. I think this could be streamlined, or I'm missing something 
### Workaround, set "reset_hooks_end = False" in above and then run. The hooks will reset by default after "run_with_cache"


out, cache2 = model.run_with_cache(torch.tensor(5.))
cache2

layer2.hook_square
Output after intervening on layer2.hook_scaled -4.0
layer2.hook_square


{'hook_in': tensor(5.),
 'layer1.hook_square': tensor(25.),
 'hook_mid': tensor(28.),
 'layer2.hook_square': tensor(0.),
 'hook_out': tensor(-4.)}

TRANSFORMER MODELS: 
Instead of the toy model, use a gpt-2 transformer. There are many currently accessible by easy transformer 

In [47]:
# Load GPT-2 small 

model_name = 'gpt2' #@param ['gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl', 'facebook/opt-125m', 'facebook/opt-1.3b', 'facebook/opt-2.7b', 'facebook/opt-6.7b', 'facebook/opt-13b', 'facebook/opt-30b', 'facebook/opt-66b', 'EleutherAI/gpt-neo-125M', 'EleutherAI/gpt-neo-1.3B', 'EleutherAI/gpt-neo-2.7B', 'EleutherAI/gpt-j-6B', 'EleutherAI/gpt-neox-20b']
model = tl.HookedTransformer.from_pretrained(model_name)
#model = tl.EasyTransformer(model_name) #deprecated based on model defined above 
if torch.cuda.is_available():
    model.to('cuda')

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2 into HookedTransformer


In [125]:
## still confused as to when and why we should use prepend_bos = False 
prompt = 'Interpretability is great'
tokens = model.to_tokens(prompt)
#tokens = model.to_tokens(prompt,prepend_bos=False)

prompt_2 = 'AI Alignment is great'
tokens_2 = model.to_tokens(prompt_2)
#tokens_2 = model.to_tokens(prompt_2,prepend_bos=False)
def show_tokens(tokens):
    # Prints the tokens as text, separated by |
    if type(tokens)==str:
        # If we input text, tokenize first
        tokens = model.to_tokens(tokens,prepend_bos=False)
    text_tokens = [model.tokenizer.decode(t) for t in tokens.squeeze()]
    print('|'.join(text_tokens))
show_tokens(tokens)
show_tokens(tokens_2)

<|endoftext|>|Inter|pret|ability| is| great
<|endoftext|>|AI| Al|ignment| is| great


In [126]:
# reset the hooks from the last example! This is already done, but just in case?
model.reset_hooks()
# try running with cache 
original_logits = model(tokens)
### get_corner (in utils) gets top corner of logits up to specified dim (could print all of it)
print('Top corner of logits')
print(get_corner(original_logits, 5))

Top corner of logits
tensor([[[ 7.5261, 11.1214,  7.8919,  8.1297,  7.4282],
         [ 4.5895,  4.6344,  3.1930,  3.0397,  4.7588],
         [ 6.0709,  6.6744,  2.5337,  0.9526,  3.9620],
         [ 7.8493,  9.2531,  5.9004,  2.1225,  3.5641],
         [ 4.2368,  4.4603,  1.5758, -0.1628,  2.1362]]],
       grad_fn=<SliceBackward0>)


In [127]:
print('Reference: Hyperparameters for the model')
dataclasses.asdict(model.cfg)

Reference: Hyperparameters for the model


{'n_layers': 12,
 'd_model': 768,
 'n_ctx': 1024,
 'd_head': 64,
 'model_name': 'gpt2',
 'n_heads': 12,
 'd_mlp': 3072,
 'act_fn': 'gelu_new',
 'd_vocab': 50257,
 'eps': 1e-05,
 'use_attn_result': False,
 'use_attn_scale': True,
 'use_split_qkv_input': False,
 'use_local_attn': False,
 'original_architecture': 'GPT2LMHeadModel',
 'from_checkpoint': False,
 'checkpoint_index': None,
 'checkpoint_label_type': None,
 'checkpoint_value': None,
 'tokenizer_name': 'gpt2',
 'window_size': None,
 'attn_types': None,
 'init_mode': 'gpt2',
 'normalization_type': 'LNPre',
 'device': 'cpu',
 'attention_dir': 'causal',
 'attn_only': False,
 'seed': None,
 'initializer_range': 0.02886751345948129,
 'init_weights': False,
 'scale_attn_by_inverse_layer_idx': False,
 'positional_embedding_type': 'standard',
 'final_rms': False,
 'd_vocab_out': 50257,
 'parallel_attn_mlp': False,
 'rotary_dim': None,
 'n_params': 84934656,
 'use_hook_tokens': False}

USING THE MODEL 
- can input either tokens or text, which is converted to tokens
- This one differs from the hugging face model because it lacks the bias term in the unembedding. This cancels out after taking the softmax, so while the logits are different the log probs are (largely) the same 

In [128]:
logits_tokens = model(tokens)
logits_text = model(prompt)

original_model = AutoModelForCausalLM.from_pretrained(model_name)
# easy_logits = model(tokens).cpu()
original_model_logits = original_model(tokens).logits

easy_logits = model(prompt).cpu()

easy_log_probs = F.log_softmax(easy_logits, dim=-1)
original_model_log_probs = F.log_softmax(original_model_logits, dim=-1)

print('Fraction of log probs the same between easy model and original model:')
print(torch.isclose(original_model_log_probs, easy_log_probs).sum()/easy_log_probs.numel())
print('Fraction of logits the same between easy model and original model:')
print(torch.isclose(original_model_logits, easy_logits).sum()/easy_logits.numel())

Fraction of log probs the same between easy model and original model:
tensor(0.9996)
Fraction of logits the same between easy model and original model:
tensor(0.)


Print the shapes of all activations (that are accessible by each hook)
- A useful reference for creating hooks
- The shapes of each activation should be (batch, position)

In [60]:
# make a function that grabs the activation of that name (to grab all hooks)
all_hooks_fn = lambda name: True
## make a function that shows what to do with the hooks (in this case, display)
def print_shape(tensor, hook):
    print(f"Activation at hook {hook.name} has shape {tensor.shape}")
# Is there any reason why they chose such high token numbers? 
rand_toks = torch.randint(1000,10000,(4,50)) #(batch size, seq length)
print(rand_toks.shape)
logits = model.run_with_hooks(rand_toks, fwd_hooks = [(all_hooks_fn, print_shape)])

## can use this to check the dimensions of each activation in the network (very useful!)

torch.Size([4, 50])
Activation at hook hook_embed has shape torch.Size([4, 50, 768])
Activation at hook hook_pos_embed has shape torch.Size([4, 50, 768])
Activation at hook blocks.0.hook_resid_pre has shape torch.Size([4, 50, 768])
Activation at hook blocks.0.ln1.hook_scale has shape torch.Size([4, 50, 1])
Activation at hook blocks.0.ln1.hook_normalized has shape torch.Size([4, 50, 768])
Activation at hook blocks.0.ln1.hook_scale has shape torch.Size([4, 50, 1])
Activation at hook blocks.0.ln1.hook_normalized has shape torch.Size([4, 50, 768])
Activation at hook blocks.0.ln1.hook_scale has shape torch.Size([4, 50, 1])
Activation at hook blocks.0.ln1.hook_normalized has shape torch.Size([4, 50, 768])
Activation at hook blocks.0.attn.hook_q has shape torch.Size([4, 50, 12, 64])
Activation at hook blocks.0.attn.hook_k has shape torch.Size([4, 50, 12, 64])
Activation at hook blocks.0.attn.hook_v has shape torch.Size([4, 50, 12, 64])
Activation at hook blocks.0.attn.hook_attn_scores has sha

print the top corner of activations: 
- Can use these to compare any altered activations to the original ones, without looking through the full matrix 


In [66]:
#uses default size of 3x3 corner
### colab used "tokens" from earlier, I'm going to stick with rand_toks
def print_corner(tensor, hook):
    print(hook.name)
    print(get_corner(tensor))

logits = model.run_with_hooks(rand_toks, fwd_hooks=[(all_hooks_fn, print_corner)])



hook_embed
tensor([[[-0.1213,  0.0433,  0.1574],
         [-0.0957,  0.0820,  0.1314],
         [ 0.0227, -0.0485,  0.1109]],

        [[ 0.2166, -0.1912, -0.0205],
         [ 0.0187, -0.1366,  0.0230],
         [-0.0300, -0.0031,  0.1593]],

        [[ 0.0905,  0.0264,  0.1115],
         [ 0.1901, -0.0444,  0.0603],
         [ 0.1749, -0.1153,  0.2472]]], grad_fn=<SliceBackward0>)
hook_pos_embed
tensor([[[-0.0134, -0.1920,  0.0095],
         [ 0.0250, -0.0528, -0.0939],
         [ 0.0065, -0.0825,  0.0568]],

        [[-0.0134, -0.1920,  0.0095],
         [ 0.0250, -0.0528, -0.0939],
         [ 0.0065, -0.0825,  0.0568]],

        [[-0.0134, -0.1920,  0.0095],
         [ 0.0250, -0.0528, -0.0939],
         [ 0.0065, -0.0825,  0.0568]]], grad_fn=<SliceBackward0>)
blocks.0.hook_resid_pre
tensor([[[-0.1347, -0.1487,  0.1669],
         [-0.0707,  0.0292,  0.0375],
         [ 0.0291, -0.1310,  0.1676]],

        [[ 0.2032, -0.3832, -0.0110],
         [ 0.0437, -0.1894, -0.0708],
         [

In [73]:
## cache all the activations 
model.reset_hooks()
logits, cache = model.run_with_cache(rand_toks)
for name in cache:
    print(name, cache[name].shape)

# This should be the same as printing the shape of activations like we did before. Let's check: 
model.reset_hooks()
shape_cache = {}
def cache_shape(tensor, hook):
    shape_cache[hook.name] = tensor.shape
logits = model.run_with_hooks(rand_toks, fwd_hooks = [(all_hooks_fn, cache_shape)])

for name in cache:
    assert cache[name].shape == shape_cache[name]



hook_embed torch.Size([4, 50, 768])
hook_pos_embed torch.Size([4, 50, 768])
blocks.0.hook_resid_pre torch.Size([4, 50, 768])
blocks.0.ln1.hook_scale torch.Size([4, 50, 1])
blocks.0.ln1.hook_normalized torch.Size([4, 50, 768])
blocks.0.attn.hook_q torch.Size([4, 50, 12, 64])
blocks.0.attn.hook_k torch.Size([4, 50, 12, 64])
blocks.0.attn.hook_v torch.Size([4, 50, 12, 64])
blocks.0.attn.hook_attn_scores torch.Size([4, 12, 50, 50])
blocks.0.attn.hook_pattern torch.Size([4, 12, 50, 50])
blocks.0.attn.hook_z torch.Size([4, 50, 12, 64])
blocks.0.hook_attn_out torch.Size([4, 50, 768])
blocks.0.hook_resid_mid torch.Size([4, 50, 768])
blocks.0.ln2.hook_scale torch.Size([4, 50, 1])
blocks.0.ln2.hook_normalized torch.Size([4, 50, 768])
blocks.0.mlp.hook_pre torch.Size([4, 50, 3072])
blocks.0.mlp.hook_post torch.Size([4, 50, 3072])
blocks.0.hook_mlp_out torch.Size([4, 50, 768])
blocks.0.hook_resid_post torch.Size([4, 50, 768])
blocks.1.hook_resid_pre torch.Size([4, 50, 768])
blocks.1.ln1.hook_scale

EDITING ACTIVATIONS

In [129]:
##### PRUNING ATTENTION HEADS ###########

head_dict = {3:[0,3,7], 7:[8,9]}
fwd_hooks = []
for layer, heads in head_dict.items():
    hook = f"blocks.{layer}.attn.hook_z" #this can be a string
    hook_fn = f"prune_fn_{layer}" #this needs to be a callable function of this name
    fwd_hooks.append((hook, eval(hook_fn)))


logits = model.run_with_hooks(tokens, fwd_hooks = fwd_hooks)

logits

tensor([[[ 7.4487, 11.0131,  7.8047,  ..., -2.9968, -3.2080,  8.5503],
         [ 4.5501,  4.6872,  3.5226,  ..., -4.7606, -2.3688,  4.5484],
         [ 6.0779,  6.6369,  2.5403,  ..., -3.2799, -1.3250,  7.7766],
         [ 7.9233,  9.0306,  5.9391,  ...,  1.1033, -1.2689,  8.0923],
         [ 4.1626,  4.4285,  1.5028,  ..., -1.7751, -1.4225,  3.4835],
         [14.3212,  8.7445,  3.4807,  ...,  1.2178, -2.7917,  9.4326]]],
       grad_fn=<AddBackward0>)

In [144]:
##### RESTRICT ALL ATTENTION HEADS TO ONLY ATTEND TO CURRENT AND PREVIOUS TOKENS ###########

## This is a separate example from above. Always reset hooks (even though the default of run_with_hooks and run_with_cache is to do it anyway)
### Also, I think they use the cache just for the original model, to compare with the hooked function. We could easily make the function to change hooks update the cache. 

model.reset_hooks()
def filter_hook_attn(name):
    split_name = name.split('.')
    #if (split_name[-1] == 'hook_attn'):
    #print(split_name[-1])
    ## only change the attn heads
    return (split_name[-1] == 'hook_pattern')

# Apply a mask to the attention pattern that only considers the current and previous tokens
### attn has shape batch x head_no x query_pos x key_pos
def restrict_attn(attn, hook):
    # key and query length (context length) should be the same? 
    assert attn.size(-2) == attn.size(-1)
    n_context = attn.size(-1)
    key_pos = torch.arange(n_context)[None, :]
    query_pos = torch.arange(n_context)[:,None]
    ZERO = torch.tensor(0.)
    mask = (key_pos > (query_pos - 2))
    
    if torch.cuda.is_available():
        ZERO = ZERO.cuda()
        mask = mask.cuda()
    attn = torch.where(mask, attn, ZERO)
    #print(attn)
    return attn

logits = model.run_with_hooks(tokens, fwd_hooks = [(filter_hook_attn, restrict_attn)])

print(f"New logits = {get_corner(logits,5)}")
print(f"OG logits = {get_corner(original_logits,5)}")
logits - original_logits, logits.shape



New logits = tensor([[[ 7.5261, 11.1214,  7.8919,  8.1297,  7.4282],
         [ 4.5895,  4.6344,  3.1930,  3.0397,  4.7588],
         [ 7.4953,  8.5096,  4.7810,  5.4829,  6.5468],
         [ 9.6950, 13.1113,  6.2229,  3.7978,  6.4755],
         [ 4.9499,  6.9523,  0.6289,  0.3997,  1.7647]]],
       grad_fn=<SliceBackward0>)
OG logits = tensor([[[ 7.5261, 11.1214,  7.8919,  8.1297,  7.4282],
         [ 4.5895,  4.6344,  3.1930,  3.0397,  4.7588],
         [ 6.0709,  6.6744,  2.5337,  0.9526,  3.9620],
         [ 7.8493,  9.2531,  5.9004,  2.1225,  3.5641],
         [ 4.2368,  4.4603,  1.5758, -0.1628,  2.1362]]],
       grad_fn=<SliceBackward0>)


(tensor([[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 1.4244,  1.8352,  2.2473,  ..., -0.9822, -0.6487, -0.6475],
          [ 1.8457,  3.8582,  0.3225,  ..., -0.3913,  0.0780,  0.3606],
          [ 0.7131,  2.4920, -0.9469,  ...,  0.1432,  2.0542, -0.8026],
          [-0.0442,  1.6058, -2.9549,  ..., -0.7779,  0.7996, -1.3249]]],
        grad_fn=<SubBackward0>),
 torch.Size([1, 6, 50257]))

FREEZING ATTENTION PATTERNS: 
Run the model twice: Once on the original text, getting the attention pattern, again on the new text, replacing them with the cached pattern 
-- I'll do it once the way they did (deprecated), and again the way I think it should be done now 

In [158]:
## old way 
attn_cache = {}
def cache_attn(attn, hook):
    attn_cache[hook.name] = attn


def freeze_attn(attn, hook):
    #print(attn_cache[hook.name])
    return attn_cache[hook.name]

logits_0 = model.run_with_hooks(tokens, fwd_hooks = [(filter_hook_attn, cache_attn)])
print(attn_cache.keys())
logits_1 = model.run_with_hooks(tokens_2, fwd_hooks = [(filter_hook_attn, freeze_attn)])

logits_0, logits_1, logits_0.shape, logits_1.shape


dict_keys(['blocks.0.attn.hook_pattern', 'blocks.1.attn.hook_pattern', 'blocks.2.attn.hook_pattern', 'blocks.3.attn.hook_pattern', 'blocks.4.attn.hook_pattern', 'blocks.5.attn.hook_pattern', 'blocks.6.attn.hook_pattern', 'blocks.7.attn.hook_pattern', 'blocks.8.attn.hook_pattern', 'blocks.9.attn.hook_pattern', 'blocks.10.attn.hook_pattern', 'blocks.11.attn.hook_pattern'])


(tensor([[[ 7.5261, 11.1214,  7.8919,  ..., -3.1299, -3.3873,  8.5934],
          [ 4.5895,  4.6344,  3.1930,  ..., -4.6853, -2.3474,  4.6327],
          [ 6.0709,  6.6744,  2.5337,  ..., -3.2367, -1.5178,  8.0657],
          [ 7.8493,  9.2531,  5.9004,  ...,  0.9130, -1.4072,  8.2276],
          [ 4.2368,  4.4603,  1.5758,  ..., -1.7220, -1.1896,  3.2500],
          [14.2398,  8.7174,  4.0264,  ...,  1.2469, -2.7178,  9.8876]]],
        grad_fn=<AddBackward0>),
 tensor([[[ 7.5261e+00,  1.1121e+01,  7.8919e+00,  ..., -3.1299e+00,
           -3.3873e+00,  8.5934e+00],
          [ 6.1054e+00,  6.3903e+00,  4.8687e+00,  ..., -8.8158e-01,
            7.6134e-02,  6.2353e+00],
          [ 6.2097e+00,  5.8213e+00,  4.8398e+00,  ..., -4.5794e+00,
            1.4583e-03,  5.5054e+00],
          [ 8.3301e+00,  7.7019e+00,  5.6676e+00,  ..., -2.2781e+00,
           -3.6617e-01,  7.6546e+00],
          [ 5.0353e+00,  4.3325e+00,  2.1272e+00,  ..., -4.2344e+00,
           -1.5338e+00,  3.3912e+00]

In [162]:
### New way: I don't think it matters that you filter the original model, since it will get filtered out when we do it again 

logits_OG, cache_OG = model.run_with_cache(tokens)

def freeze_attn_new(attn, hook):
    #print(hook.name)
    return cache_OG[hook.name]

logits_new = model.run_with_hooks(tokens_2, fwd_hooks = [(filter_hook_attn, freeze_attn_new)])

print(all(einops.rearrange((logits_OG - logits_0)==0, "b s v -> (b s v)")))
print(all(einops.rearrange((logits_1 - logits_new)==0, "b s v -> (b s v)")))



True
True
